# Préparation des données

>Ce livret prépare les matrices de données (données d'entrainement et données de test).

## But
* Uniformiser les fichiers de données
* Générer les fichiers binaires pour les données

<h1>Table des matières<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Préparation-des-données" data-toc-modified-id="Préparation-des-données-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Préparation des données</a></span><ul class="toc-item"><li><span><a href="#But" data-toc-modified-id="But-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>But</a></span></li></ul></li><li><span><a href="#Mettre-en-place-l'environnement" data-toc-modified-id="Mettre-en-place-l'environnement-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Mettre en place l'environnement</a></span></li><li><span><a href="#Uniformiser-les-données" data-toc-modified-id="Uniformiser-les-données-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Uniformiser les données</a></span></li><li><span><a href="#Générer-la-matrice-binaire" data-toc-modified-id="Générer-la-matrice-binaire-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Générer la matrice binaire</a></span><ul class="toc-item"><li><span><a href="#Générer-plusieurs-fichiers-binaires" data-toc-modified-id="Générer-plusieurs-fichiers-binaires-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Générer plusieurs fichiers binaires</a></span></li></ul></li></ul></div>

# Mettre en place l'environnement

In [1]:
# Utilités
import os

import numpy as np
import pandas as pd
import scipy.io

In [2]:
# os.chdir("../") # Dossier précédent
os.chdir("./")
os.getcwd()     # Vérifier le présent dossier

'C:\\Users\\amass\\OneDrive\\02_Education\\02_Maitrise\\Cours\\PROJET_MAITRISE\\application'

In [3]:
param_livret = {
    # Spécifique aux données
    "dossier": "data",
    "sousdossier": "simulation_simple",
    "fichier": "SimSimple",
    "fichier_sortie": "SimSimple",
    "format_size": 1.0,
    "retourner": True
}

In [4]:
param_binaire = {
    # Spécifique aux données
    "dossier": "data",
    "sousdossier": "simulation_simple",
    "fichier": "SimSimple_100",
    "format_size": 1.0,
    "retourner": True,
    # Spécifique pour la matrice binaire
    "p_manquants": 0.1, 
    "multiple": True
}

# Uniformiser les données

In [5]:
def formatage_donnees(param_livret):
    # Initialisation des variables
    dossier = param_livret["dossier"]
    sousdossier = param_livret["sousdossier"]
    nom_fichier_donnees = param_livret["fichier"]
    nom_fichier_sortie = param_livret["fichier_sortie"]
    format_size = param_livret["format_size"]
    retourner = param_livret["retourner"]
    path = os.getcwd()
    
    # Chargement de la matrice
    ch_fichier_donnees = os.path.join(path, dossier, sousdossier, nom_fichier_donnees)
    donnees = scipy.io.loadmat(ch_fichier_donnees)
    donnees = donnees[list(donnees)[3]]
    
    # Si c'est un tenseur, le transformer en matrice
    dimensions = donnees.shape
    
    if len(dimensions) > 2:
        dim1 = dimensions[0]
        donnees = donnees.reshape(dim1, -1)
    
    # Si l'utilisateur spécifie un % de données à utiliser, l'appliquer.
    dim1, dim2 = donnees.shape
    format_size = np.round(format_size, decimals=2)
    
    if format_size > 1.0:
        format_size = 1.0
    
    dim1 = int(np.round(dim1 * format_size))
    dim2 = int(np.round(dim2 * format_size))
    donnees = donnees[:dim1, :dim2]
    
    # Sauvegarder le fichier
    nom_format = str(int(format_size * 100))
    nom_fichier = "{:}_{:}.mat".format(os.path.join(dossier, sousdossier, nom_fichier_sortie), nom_format)
    
    scipy.io.savemat(nom_fichier, {"mat": donnees})
    return donnees if retourner == True else None

In [6]:
# Appel de la fonction   
matrice = formatage_donnees(param_livret)

# Générer la matrice binaire

In [7]:
def generer_matrice_binaire(param_binaire):
    dossier = param_binaire["dossier"]
    sousdossier = param_binaire["sousdossier"]
    nom_fichier = param_binaire["fichier"]
    p_manquants = param_binaire["p_manquants"]
    retourner = param_binaire["retourner"]
    
    path = os.getcwd()
    
    ch_fichier_donnees = os.path.join(path, dossier, sousdossier, nom_fichier)
    
    donnees = scipy.io.loadmat(ch_fichier_donnees)
    donnees = donnees["mat"]
    # Générer la matrice binaire
    p_manquants = np.round(p_manquants, decimals=2)
    
    if p_manquants > 1.0:
        p_manquants = 1.0
    
    p0 = p_manquants
    p1 = 1 - p0
    
    dim1, dim2 = donnees.shape
    
    np.random.seed(2020)
    matrice_binaire = np.random.choice([0, 1], size=(dim1, dim2), p=[p0, p1])
    
    # Sauvegarder la matrice binaire
    nom_proportion = str(int(p0 * 100))
    
    
    ch_matrice_binaire = os.path.join(dossier, sousdossier, nom_fichier)
    nom_fichier = "{:}_{:}.mat".format(ch_matrice_binaire, nom_proportion)
    scipy.io.savemat(nom_fichier, {"mat": matrice_binaire})
    
    
    # Message pour l'utilisateur
    prop_finale = len(np.where(matrice_binaire == 0)[0]) / (dim1 * dim2)
    prop_finale = prop_finale * 100
    
    print("La proportion de zéro dans la matrice binaire est de {:2.2f} %".format(prop_finale))
    
    return matrice_binaire if retourner == True else None

In [8]:
matrice_binaire = generer_matrice_binaire(param_binaire = param_binaire)

La proportion de zéro dans la matrice binaire est de 10.28 %


## Générer plusieurs fichiers binaires

In [9]:
multiple = param_binaire["multiple"]


if multiple == True:
    print("Création de multiples fichiers binaires")
    prop_manquants = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]

    for p in prop_manquants:
        print("Proportion a générer: {:2.0f} %".format(p * 100))
        param_binaire_multiple = param_binaire.copy()
        param_binaire_multiple["p_manquants"] = p
        matrice_binaire = generer_matrice_binaire(param_binaire_multiple)
else:
    print("Création d'un seul fichier binaire")
    matrice_binaire = generer_matrice_binaire(param_binaire)

Création de multiples fichiers binaires
Proportion a générer: 10 %
La proportion de zéro dans la matrice binaire est de 10.28 %
Proportion a générer: 20 %
La proportion de zéro dans la matrice binaire est de 20.27 %
Proportion a générer: 30 %
La proportion de zéro dans la matrice binaire est de 30.45 %
Proportion a générer: 40 %
La proportion de zéro dans la matrice binaire est de 40.20 %
Proportion a générer: 50 %
La proportion de zéro dans la matrice binaire est de 49.89 %
Proportion a générer: 60 %
La proportion de zéro dans la matrice binaire est de 60.05 %
Proportion a générer: 70 %
La proportion de zéro dans la matrice binaire est de 70.23 %
Proportion a générer: 80 %
La proportion de zéro dans la matrice binaire est de 80.19 %
Proportion a générer: 90 %
La proportion de zéro dans la matrice binaire est de 90.10 %
